In [1]:
import numpy as np
import scipy as scp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import mpmath
from mpmath import mp

from numba import jit
from numba import cuda

import os
import glob
import time
import IPython.display

import itertools

from tqdm.notebook import tqdm

from lat_class_parallel_map import *
"""
from tqdm.notebook import tqdm
"""

'\nfrom tqdm.notebook import tqdm\n'

In [2]:
"""determine weights"""

mp.prec=100

eta = 4.01
lam = 1.000
mu = 1.050

n_min = 0
n_max = 202

num_W_val = (n_max - n_min)//2
"""
#W = []
#W_obs_ratio = []

W = np.zeros(num_W_val)
W_obs_ratio = np.zeros(num_W_val)

def weight_integrand(r, n, e, l):
    return mpmath.power(r, (n+1)) * mpmath.exp(- e * r ** 2 - l * r ** 4)

for n in tqdm(range(n_min, n_max, 2)):
    weight_integral = mpmath.quad(lambda x:weight_integrand(x,n,eta,lam), [0, mpmath.inf])
    #W.append(weight_integral)
    W[n//2] = weight_integral
    obs_ratio_integral =  mpmath.quad(lambda x:weight_integrand(x,n+2,eta,lam), [0, mpmath.inf])/mpmath.quad(lambda x:weight_integrand(x,n,eta,lam), [0, mpmath.inf])
    #W_obs_ratio.append(obs_ratio_integral)
    W_obs_ratio[n//2] = obs_ratio_integral
"""

'\n#W = []\n#W_obs_ratio = []\n\nW = np.zeros(num_W_val)\nW_obs_ratio = np.zeros(num_W_val)\n\ndef weight_integrand(r, n, e, l):\n    return mpmath.power(r, (n+1)) * mpmath.exp(- e * r ** 2 - l * r ** 4)\n\nfor n in tqdm(range(n_min, n_max, 2)):\n    weight_integral = mpmath.quad(lambda x:weight_integrand(x,n,eta,lam), [0, mpmath.inf])\n    #W.append(weight_integral)\n    W[n//2] = weight_integral\n    obs_ratio_integral =  mpmath.quad(lambda x:weight_integrand(x,n+2,eta,lam), [0, mpmath.inf])/mpmath.quad(lambda x:weight_integrand(x,n,eta,lam), [0, mpmath.inf])\n    #W_obs_ratio.append(obs_ratio_integral)\n    W_obs_ratio[n//2] = obs_ratio_integral\n'

In [3]:
W_obs_ratio = np.loadtxt("W_obs_ratio.txt")
W = np.loadtxt("W.txt")

In [4]:
kl_sim = lattice_sim()
kl_sim.mu = mu

[2 2 3 3]
Setting worm to tail: [[1 2]], head: [[[1 2]
  [1 2]]]

with k_delta: [1]

white sites: 5
black sites: 4


In [5]:
#seed = 42
#np.random.seed(seed)

In [ ]:
phi2_obs_file = open("phi2_obs.txt","w")
n_obs_file = open("n_obs.txt","w")



num_iter = 50

iter_per_obs = 5

worms_per_iter = 1

mcmc_per_iter = 1

phi2s = np.zeros(shape=(num_iter//iter_per_obs))
ns = np.zeros(shape=(num_iter//iter_per_obs))

i_obs = 0

print("starting worm k algorithm")

for i in tqdm(range(num_iter)):
    
    ##worm algorithm k sampling
    ##count the number of worms for one iterations
    worm_count = 0
    
    kl_sim.reset_worm_i_worm(0,
                   kl_sim.k_links, kl_sim.l_links, kl_sim.mu, kl_sim.W, kl_sim.f,
                   kl_sim.old_head_lat, kl_sim.new_head_lat, kl_sim.tail_df_lat,
                   kl_sim.move_dim, kl_sim.move_sign, kl_sim.move_i, kl_sim.move_set, kl_sim.moves,
                   kl_sim.head, kl_sim.tail, kl_sim.worm_link_coord, kl_sim.worm_closed,
                   kl_sim.k_delta, kl_sim.dk, kl_sim.df, kl_sim.df0,
                   kl_sim.p_acc,
                   kl_sim.num_worms, kl_sim.lat_size, kl_sim.dim

    )

    ##limit the number of worms in one iteration
    while worm_count < worms_per_iter:
        worm_closed = False
        
        print("started worm")
        ##let the worm run AND close in this loop
        
        kl_sim.run_i_worm(0,
                       kl_sim.k_links, kl_sim.l_links, kl_sim.mu, kl_sim.W, kl_sim.f,
                       kl_sim.old_head_lat, kl_sim.new_head_lat, kl_sim.tail_df_lat,
                       kl_sim.move_dim, kl_sim.move_sign, kl_sim.move_i, kl_sim.move_set, kl_sim.moves,
                       kl_sim.head, kl_sim.tail, kl_sim.worm_link_coord, kl_sim.worm_closed,
                       kl_sim.k_delta, kl_sim.dk, kl_sim.df, kl_sim.df0,
                       kl_sim.p_acc,
                       kl_sim.num_worms, kl_sim.lat_size, kl_sim.dim
    
                         )
        
        
        print("worm has closed")
        worm_count += 1
        
    print("ended worm k algorithm")
    ##mcmc l sweep
    print("\n")
    print("starting mcmc l sweep")
    l_sweep = 0
    while l_sweep < mcmc_per_iter:
        
        ##sample white sites
        kl_sim.sweep(
              kl_sim.w_sites, kl_sim.w_target_sites, kl_sim.n_w_sites,
              kl_sim.k_links, kl_sim.l_links, kl_sim.W, kl_sim.f,
              kl_sim.w_link_coord, kl_sim.mcmc_w_mod_dim, kl_sim.mcmc_w_dl, kl_sim.mcmc_w_df, kl_sim.mcmc_w_target_df,
              kl_sim.move_set, kl_sim.w_moves,
              kl_sim.lat_size, kl_sim.dim,
              kl_sim.mcmc_w_p_acc
             )

        ##sample black sites
        kl_sim.sweep(
              kl_sim.b_sites, kl_sim.b_target_sites, kl_sim.n_b_sites,
              kl_sim.k_links, kl_sim.l_links, kl_sim.W, kl_sim.f,
              kl_sim.b_link_coord, kl_sim.mcmc_b_mod_dim, kl_sim.mcmc_b_dl, kl_sim.mcmc_b_df, kl_sim.mcmc_b_target_df,
              kl_sim.move_set, kl_sim.b_moves,
              kl_sim.lat_size, kl_sim.dim,
              kl_sim.mcmc_b_p_acc
             )
        
        l_sweep += 1
        
            
    #print(f"f: {kl_sim.f}")
    print("ended mcmc l sweep")
    
    input("Press Enter to continue...")
    time.sleep(1.0)
    IPython.display.clear_output(wait=True)
    
    if i%iter_per_obs == 0:
        
        ##update observables
        ##phi2
        phi2 = 0.
        ##n
        n = 0.
        
        for coord in itertools.product(kl_sim.lat_size):
            phi2 += W_obs_ratio[kl_sim.f[tuple(coord)]]
            n += kl_sim.k_links[tuple(np.concatenate([0,coord]))]
            
        phi2 = phi2/kl_sim.num_sites
        n = n/kl_sim.num_sites
        
        #phi2_obs_file.write(phi2)
        #n_obs_file.write(n)
        
        phi2s[i_obs] = phi2
        n2s[i_obs] = n2
        
        i_obs += 1

starting worm k algorithm


resetting worm
proposing move_dim:
 [0]
proposing sign:
 [0]
proposing move_i:
 [0]
lat_coord before bc: 
 [2 2]
lat_coord after bc: 
 [2 2]
proposing new heads:
 [[3 2]]
lat_coord before bc: 
 [3 2]
lat_coord after bc: 
 [0 2]
proposing new heads:
 [[0 2]]
k_old: 0
dk: [1]
df: [2]
Resetting bits at [0 2] and [2 2]
lat_coord before bc: 
 [3 2]
lat_coord after bc: 
 [0 2]
worm closed: True
worm closed: False
Resetting worm 0 to tail: [2 2], head: [0 2]

with k_delta: 1

started worm
proposing move_dim:
 [1]
proposing sign:
 [1]
proposing move_i:
 [3]
lat_coord before bc: 
 [0 1]
lat_coord after bc: 
 [0 1]
proposing new heads:
 [[0 1]]
lat_coord before bc: 
 [0 1]
lat_coord after bc: 
 [0 1]
proposing new heads:
 [[0 1]]
k_old: 0
dk: [-1]
df: [2]
p_acc: 0.07303192906843052
p_draw: 0.11795892071928049
accepted: 0
lat_coord before bc: 
 [0 2]
lat_coord after bc: 
 [0 2]
head: [0 2], tail: [2 2]
worm closed: False
worm closed: False
proposing move_dim:
 [0]
proposing sign:
 [0]
proposing m

In [ ]:
"""
f at new site is not updated
problematic if f close to 0
"""

In [ ]:
"""
What is missing?
Save df0 when starting worms

combine i_worm loops for more efficiency and parallelization
refrain from exclusive python functionality as much as possible
Set df correctly
Check for tails and accomodate for df0
f update is not working (indexing problem?)
p is not correctly drawn?
if n_iter is exceeded, remove worms and shrink the worm arrays
"""

        ##num_worms
        ##head
        ##tail
        ##k_delta
        ##df0
        
        ##worm_closed
        ##starting
        
        ##reset by counting collisions
        ##collisions
        ##hoc_dist_ij
        ##lc_dist_ij
        
        ##reset by proposing new moves
        ##move_dim
        ##move_sign
        ##move_i
        ##moves
        ##worm_link_coord
        ##dk
        ##df
        
        ##reset by sampling
        """not an attribute?"""
        ##p